In [2]:
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.image as image
import numpy as np
import tensorflow as tf

In [3]:
from keras.datasets import cifar10
from keras.callbacks import ReduceLROnPlateau

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images.astype("float32") / 255 
test_images = test_images.astype("float32") / 255

In [4]:
def resize_images(image):
	new_size = (64, 64)
	image = tf.image.resize(image, new_size)
	return image

# 데이터 증강 추가
data_generator = ImageDataGenerator(
	preprocessing_function=resize_images,
    rotation_range=20,  # 회전
    width_shift_range=0.2,  # 좌우 이동
    height_shift_range=0.2,  # 상하 이동
    brightness_range=(0.5, 1.0),  # 밝기 조절 0~1 사이, 0에 가까울수록 어두움
    horizontal_flip=True,  # 좌우 반전
    vertical_flip=True,  # 상하 반전
	
    # rescale=1/255.0, # 학습시 적용, 시각화를 위해 임시로 주석처리
	
)

train_images_resized = resize_images(train_images)

test_images_resized = resize_images(test_images)



2024-08-20 10:50:37.552463: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-08-20 10:50:37.552631: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-08-20 10:50:37.552643: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-08-20 10:50:37.553017: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-20 10:50:37.553593: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [29]:
# batch_size = 64

# # 증강된 이미지 생성
# augmented_data_generator = data_generator.flow(
#     train_images,
#     train_labels,
#     batch_size=batch_size,
# )


# # # 증강된 첫 번째 배치 이미지 확인
# augmented_images, augmented_labels = next(augmented_data_generator)

# print(augmented_images.shape)


In [4]:
conv_base = keras.applications.vgg16.VGG16(
	include_top=False,
	input_shape=(32, 32, 3)
)


# conv_base = keras.applications.vgg16.VGG16(
# 	include_top=False,
# 	input_shape=(64, 64, 3)
# )

conv_base.trainable = False

conv_base.summary()

2024-08-20 16:33:25.593178: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-08-20 16:33:25.593257: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-08-20 16:33:25.593280: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-08-20 16:33:25.593304: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-20 16:33:25.593329: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [7]:
from tensorflow.keras import layers, models
from keras.layers import Flatten, Dense, Dropout

# inputs = keras.Input(shape=(64, 64, 3))
inputs = keras.Input(shape=(32, 32, 3))
x = conv_base(inputs, training=False)
# x = Flatten()(x)
x = layers.GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [59]:
len(train_images) // batch_size

781

In [9]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

callbacks = [keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction_cifal10.keras",
      save_best_only=True,
      monitor="val_loss")]

history = model.fit(
    train_images,
	train_labels,
	batch_size=64,
#     steps_per_epoch=len(train_images) // batch_size,
    epochs=50,
    validation_data=(test_images, test_labels),
    callbacks=callbacks)

Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 51ms/step - accuracy: 0.5293 - loss: 1.3566 - val_accuracy: 0.5596 - val_loss: 1.2639
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 57s 73ms/step - accuracy: 0.5803 - loss: 1.2193 - val_accuracy: 0.5747 - val_loss: 1.2173
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 70s 90ms/step - accuracy: 0.5937 - loss: 1.1693 - val_accuracy: 0.5838 - val_loss: 1.1949
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 94s 120ms/step - accuracy: 0.6057 - loss: 1.1318 - val_accuracy: 0.5873 - val_loss: 1.1855
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 90s 115ms/step - accuracy: 0.6084 - loss: 1.1208 - val_accuracy: 0.5917 - val_loss: 1.1726
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 88s 113ms/step - accuracy: 0.6202 - loss: 1.0892 - val_accuracy: 0.5912 - val_loss: 1.1644
Epoch 7/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 80s 103ms/step - accuracy: 0.6237 - loss: 1.0835 - val_accuracy: 0.5948 - val_loss: 1.1651
Epoch 8/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 72s 92ms/step - accuracy: 0.6300 - loss: 1.068

In [ ]:
test_loss, test_acc = model.evaluate(test_images_resized, test_labels)
print(f"\n테스트 정확도: {test_acc:.3f}")

In [12]:
# conv_base = keras.applications.vgg16.VGG16(
# 	include_top=False,
# 	input_shape=(32, 32, 3)
# )


conv_base = keras.applications.vgg16.VGG16(
	include_top=False,
	input_shape=(32, 32, 3)
)

conv_base.trainable = False

conv_base.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [13]:
from tensorflow.keras import layers, models
from keras.layers import Flatten, Dense, Dropout

inputs = keras.Input(shape=(32, 32, 3))
x = conv_base(inputs, training=False)
# x = Flatten()(x)
x = layers.GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [14]:
# resize 안한 버전 

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

callbacks = [keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction_cifal10.keras",
      save_best_only=True,
      monitor="val_loss")]

history = model.fit(
    train_images,
	train_labels,
	batch_size=64,
#     steps_per_epoch=len(train_images) // batch_size,
    epochs=50,
    validation_data=(test_images, test_labels),
    callbacks=callbacks)

Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 22s 27ms/step - accuracy: 0.4185 - loss: 1.6615 - val_accuracy: 0.5497 - val_loss: 1.2851
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.5719 - loss: 1.2332 - val_accuracy: 0.5641 - val_loss: 1.2455
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.5917 - loss: 1.1769 - val_accuracy: 0.5821 - val_loss: 1.2009
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.6021 - loss: 1.1421 - val_accuracy: 0.5843 - val_loss: 1.1840
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.6114 - loss: 1.1184 - val_accuracy: 0.5864 - val_loss: 1.1822
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.6153 - loss: 1.1007 - val_accuracy: 0.5926 - val_loss: 1.1687
Epoch 7/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.6241 - loss: 1.0752 - val_accuracy: 0.5934 - val_loss: 1.1642
Epoch 8/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.6320 - loss: 1.0597 - 